In [4]:
%load_ext dotenv 
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [5]:
import os
import warnings
warnings.filterwarnings('ignore')

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [7]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [9]:
conversation.predict(input="Hi, my name is Camilo")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: Hi, my name is Camilo
AI:

> Finished chain.


"Hello Camilo, it's nice to meet you as well. How can I assist you today?"

In [10]:
conversation.predict(input="Guess where I am from based on my name")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: Hi, my name is Camilo
AI: Hello Camilo, it's nice to meet you as well. How can I assist you today?
Human: Guess where I am from based on my name
AI:

> Finished chain.


"I'm sorry Camilo, but I cannot accurately guess where you are from based solely on your name. However, I can tell you that your name is of Spanish origin and is a popular name in many Spanish-speaking countries."

In [11]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: Hi, my name is Camilo
AI: Hello Camilo, it's nice to meet you as well. How can I assist you today?
Human: Guess where I am from based on my name
AI: I'm sorry Camilo, but I cannot accurately guess where you are from based solely on your name. However, I can tell you that your name is of Spanish origin and is a popular name in many Spanish-speaking countries.


In [12]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: Hi, my name is Camilo\nAI: Hello Camilo, it's nice to meet you as well. How can I assist you today?\nHuman: Guess where I am from based on my name\nAI: I'm sorry Camilo, but I cannot accurately guess where you are from based solely on your name. However, I can tell you that your name is of Spanish origin and is a popular name in many Spanish-speaking countries."}

In [13]:
memory = ConversationBufferMemory()

In [14]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [16]:
print(memory.buffer)
memory.load_memory_variables({})

Human: Hi
AI: What's up


{'history': "Human: Hi\nAI: What's up"}

In [18]:
# context keeps stacking new messages

memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: Not much, just hanging\nAI: Cool"}

## Convrsational buffer window memory

In [20]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)       

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

## Convrsational token buffer memory

In [23]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [25]:
# limits the number of tokens saved to the context.
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

## ConversationSummaryMemory

In [27]:
from langchain.memory import ConversationSummaryBufferMemory

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [28]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

In [29]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [30]:
conversation.predict(input="What would be a good demo to show?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.


"Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our machine learning algorithms and how they continuously improve the accuracy and efficiency of our AI. Would you like me to prepare a presentation for this demo?"

In [35]:
import textwrap
response = memory.load_memory_variables({})
textwrap.wrap(response["history"], 100)

["System: The human and AI engage in small talk before discussing the day's schedule. The AI informs",
 'the human of a morning meeting with the product team, time to work on the LangChain project, and a',
 'lunch meeting with a customer interested in the latest AI developments. The human asks what would be',
 "a good demo to show. AI: Based on the customer's interest in AI developments, I would suggest",
 'showcasing our latest natural language processing capabilities. We could demonstrate how our AI can',
 'understand and respond to complex language queries, and even provide personalized recommendations',
 "based on the user's preferences. Additionally, we could highlight our machine learning algorithms",
 'and how they continuously improve the accuracy and efficiency of our AI. Would you like me to',
 'prepare a presentation for this demo?']